In [ ]:
# Need to restart after:
!pip install convokit[llm]
!pip install convokit

In [3]:
import sys
import os

os.chdir('/content/temporal_belief_analysis/notebooks')
print("Changed working directory to:", os.getcwd())

# Absolute path to src directory
src_path = os.path.abspath(os.path.join(os.getcwd(), '..', 'src'))
if src_path not in sys.path:
    sys.path.insert(0, src_path)

FileNotFoundError: [Errno 2] No such file or directory: '/content/temporal_belief_analysis/notebooks'

In [3]:
import time
!pip install gdown
import zipfile
import nltk
from nltk.corpus import stopwords
from convokit import Corpus, download
import convokit
from temporal_belief.core.timeline_building import TimelineBuilder

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [92]:
# Download and unzip with python (Dataloading):
# !gdown "https://drive.google.com/file/d/1N0U_jUJlOYjdaju2FaU8p87uB22YBxJ0/view?usp=sharing" -O "/content/temporal_belief_analysis/pd_corpus_with_stances100000_chronological.zip" --fuzzy
# !gdown "https://drive.google.com/file/d/1DLFY6JLMZqNjwvNRZmhlV4-rnoQP_eyH/view?usp=sharing" -O "/content/temporal_belief_analysis/merged_corpus_checkpoint_5.zip" --fuzzy
# !gdown "https://drive.google.com/file/d/1nWaj5N8nsG7u5homv_kAh4CLPDv01M_Z/view?usp=sharing" -O "/content/temporal_belief_analysis/pd_corpus_with_topics.zip" --fuzzy
# !gdown "https://drive.google.com/file/d/15NMRXEkGRoGjK6TXFBHIMOPjkTyZ0keP/view?usp=sharing" -O "/content/temporal_belief_analysis/pd_corpus_with_stances200000_llm.zip" --fuzzy
# !gdown "https://drive.google.com/file/d/15nVf6Js0KsDxA9HaB0zCXhc8VdcC-Fy-/view?usp=sharing" -O "/content/temporal_belief_analysis/pd_corpus_with_stances75000_llm.zip" --fuzzy
!gdown "https://drive.google.com/file/d/1dOUvQmtjFrXq0hJvnOsP_ZLqdGEyOVNJ/view?usp=sharing" -O "/content/temporal_belief_analysis/pd_corpus_with_stances100000_llm.zip" --fuzzy

# zipfile.ZipFile("/content/temporal_belief_analysis/pd_corpus_with_stances100000_chronological.zip").extractall("/content/temporal_belief_analysis")
# zipfile.ZipFile("/content/temporal_belief_analysis/merged_corpus_checkpoint_5.zip").extractall("/content/temporal_belief_analysis")
# zipfile.ZipFile("/content/temporal_belief_analysis/pd_corpus_with_topics.zip").extractall("/content/temporal_belief_analysis")
# zipfile.ZipFile("/content/temporal_belief_analysis/pd_corpus_with_stances200000_llm.zip").extractall("/content/temporal_belief_analysis")
# zipfile.ZipFile("/content/temporal_belief_analysis/pd_corpus_with_stances75000_llm.zip").extractall("/content/temporal_belief_analysis")
zipfile.ZipFile("/content/temporal_belief_analysis/pd_corpus_with_stances100000_llm.zip").extractall("/content/temporal_belief_analysis")

Downloading...
From (original): https://drive.google.com/uc?id=1dOUvQmtjFrXq0hJvnOsP_ZLqdGEyOVNJ
From (redirected): https://drive.google.com/uc?id=1dOUvQmtjFrXq0hJvnOsP_ZLqdGEyOVNJ&confirm=t&uuid=69dd2b29-aada-4c9d-bfe0-88c684a8df1e
To: /content/temporal_belief_analysis/pd_corpus_with_stances100000_llm.zip
100% 834M/834M [00:09<00:00, 92.2MB/s]


In [4]:
corpus = Corpus(filename="/content/temporal_belief_analysis/pd_corpus_with_stances100000_llm")

In [59]:
# === 0) Inputs: a ConvoKit Corpus ===
# from convokit import Corpus
# corpus = Corpus("path_or_url")

STANCE2ID = {"neutral":0, "left-leaning":1, "right-leaning":2}
NORMALIZE = {
    "neutral":"neutral",
    "left":"left-leaning", "left-leaning":"left-leaning",
    "right":"right-leaning","right-leaning":"right-leaning",
}
VALID = set(STANCE2ID.keys())

utterances, labels, utt_ids, user_ids, topics = [], [], [], [], []

for u in corpus.iter_utterances():
    txt = (u.text or "").strip()
    stance = u.meta.get('detected_stance')
    main_post_id = u.conversation_id
    main_post = corpus.get_utterance(main_post_id)
    conv = corpus.get_conversation(main_post_id)

    conf_raw = u.meta.get('stance_confidence')
    # normalize confidence
    try:
        conf = float(conf_raw)
    except (TypeError, ValueError):
        conf = None

    # --------- SKIP problematic utterances ----------
    # missing/empty main post or duplicate of main post
    if not txt or txt in {'[removed]', '[deleted]', '.'}:
        continue

    if not main_post.text or txt == main_post.text:
        continue

    if 'stance_error' in u.meta.keys() and stance == 'neutral':
        continue

    if stance not in {'left-leaning', 'right-leaning', 'neutral'}:
        continue

    # confidence threshold depends on stance
    if stance == 'neutral':
        if conf is None or conf < 0.9:
            continue
    elif stance == 'left-leaning':
        if conf is None or conf < 0.9:
            continue
    elif stance == 'right-leaning':
        if conf is None or conf < 0.8:
            continue

    # --- EDIT HERE: pull topic if you have it, else use None/"" ---

    raw = u.meta.get("detected_stance")

    # ------------------------------------------------

    # Okay also check this from runpod

    if not raw:
        continue
    lab = NORMALIZE.get(str(raw).strip().lower())  # map aliases
    if lab not in VALID:                           # filters Unknown / other
        continue

    # --- EDIT HERE: pull topic if you have it, else use None/"" ---
    topic = u.meta.get("topic")  # or compute from your mapping
    # --------------------------------------------------------------

    utterances.append(txt)
    labels.append(lab)
    utt_ids.append(u.id)
    user_ids.append(u.speaker.id if u.speaker else "unknown")
    topics.append(topic)

In [2]:
# --- Cap max utterances per label ---
from collections import defaultdict
import numpy as np

# Set your per-label maximums here:
MAX_PER_LABEL = {
    "neutral": 1250,
    "left-leaning": 2250,
    "right-leaning": 2500,
}

# Shuffle indices first for randomness
all_idx = np.arange(len(labels))
rng = np.random.default_rng(42)
rng.shuffle(all_idx)

# Collect capped indices
label_counts = defaultdict(int)
keep_idx = []

for i in all_idx:
    lab = labels[i]
    if lab in MAX_PER_LABEL and label_counts[lab] < MAX_PER_LABEL[lab]:
        keep_idx.append(i)
        label_counts[lab] += 1

# Apply filtering
utterances = [utterances[i] for i in keep_idx]
labels     = [labels[i] for i in keep_idx]
utt_ids    = [utt_ids[i] for i in keep_idx]
user_ids   = [user_ids[i] for i in keep_idx]
topics     = [topics[i] for i in keep_idx]

print("Final counts per label:")
from collections import Counter
print(Counter(labels))

NameError: name 'labels' is not defined

In [67]:
# make a user-based split
import numpy as np
rng = np.random.default_rng(42)
uniq_users = np.array(sorted(set(user_ids)))
rng.shuffle(uniq_users)
cut = int(0.8 * len(uniq_users))
train_users = set(uniq_users[:cut])
val_users   = set(uniq_users[cut:])

train_idx = np.array([i for i,u in enumerate(user_ids) if u in train_users])
val_idx   = np.array([i for i,u in enumerate(user_ids) if u in val_users])

# Sanity
assert len(utterances) == len(labels) == len(utt_ids) == len(user_ids)

print(f"Kept {len(utterances)} labeled utterances.")

Kept 4800 labeled utterances.


In [68]:
from collections import Counter
import numpy as np

order = ['left-leaning', 'neutral', 'right-leaning']  # desired print order

# --- overall counts ---
all_counts = Counter(labels)
total = len(labels)
print("Overall:")
for k in order:
    c = all_counts.get(k, 0)
    print(f"  {k:13s}: {c:6d} ({c/total:.1%})")

# --- per-split counts (uses your train_idx / val_idx) ---
def counts_for_idx(idx):
    return Counter(labels[i] for i in idx)

print("\nTrain split:")
train_counts = counts_for_idx(train_idx)
for k in order:
    c = train_counts.get(k, 0)
    print(f"  {k:13s}: {c:6d} ({c/len(train_idx):.1%})")

print("\nVal split:")
val_counts = counts_for_idx(val_idx)
for k in order:
    c = val_counts.get(k, 0)
    print(f"  {k:13s}: {c:6d} ({c/len(val_idx):.1%})")

# --- optional: class weights aligned with STANCE2ID (for weighted loss) ---
num_classes = len(STANCE2ID)
class_weights = np.zeros(num_classes, dtype=np.float32)
for stance, idx in STANCE2ID.items():
    freq = all_counts.get(stance, 0)
    class_weights[idx] = (total / (num_classes * max(freq, 1)))  # inverse frequency

print("\nClass weights (index order per STANCE2ID):", class_weights)

Overall:
  left-leaning :   1800 (37.5%)
  neutral      :   1000 (20.8%)
  right-leaning:   2000 (41.7%)

Train split:
  left-leaning :   1557 (37.1%)
  neutral      :    884 (21.1%)
  right-leaning:   1757 (41.9%)

Val split:
  left-leaning :    243 (40.4%)
  neutral      :    116 (19.3%)
  right-leaning:    243 (40.4%)

Class weights (index order per STANCE2ID): [1.6       0.8888889 0.8      ]


In [69]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from torch.utils.data import Dataset
import torch, numpy as np
from transformers import EarlyStoppingCallback
from torch.utils.data import DataLoader, WeightedRandomSampler  # <-- add this
from collections import Counter
import torch
from sklearn.metrics import precision_recall_fscore_support, accuracy_score
from transformers import Trainer
import torch.nn as nn

# STANCE2ID = {"neutral":0, "left-leaning":1, "right-leaning":2}
ID2STANCE = {v:k for k,v in STANCE2ID.items()}
MAX_LEN = 128

# ==== you provide ====
# utterances: list[str]; labels: list[str] aligned with utterances
# Prefer split by USER so test utterances come from unseen users.
utterances = utterances   # texts
labels     = labels
train_idx, val_idx = train_idx, val_idx # indices for your split (ideally user-based)
# =====================

tok = AutoTokenizer.from_pretrained("distilbert-base-uncased")
model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased", num_labels=3, id2label=ID2STANCE, label2id=STANCE2ID
)

class StanceDS(Dataset):
    def __init__(self, texts, labs):
        self.enc = tok(texts, truncation=True, padding=True, max_length=MAX_LEN, return_tensors="pt")
        self.y = torch.tensor([STANCE2ID[l] for l in labs], dtype=torch.long)
    def __len__(self): return len(self.y)
    def __getitem__(self, i):
        item = {k:v[i] for k,v in self.enc.items()}
        item["labels"] = self.y[i]
        return item

train_ds = StanceDS([utterances[i] for i in train_idx], [labels[i] for i in train_idx])
val_ds   = StanceDS([utterances[i] for i in val_idx],   [labels[i] for i in val_idx])

# train_counts = Counter(labels[i] for i in train_idx)
# num_classes = len(STANCE2ID)
# total_train = len(train_idx)

# weights_np = np.zeros(num_classes, dtype=np.float32)
# for stance, idx in STANCE2ID.items():
#     freq = train_counts.get(stance, 1)
#     weights_np[idx] = total_train / (num_classes * freq)  # inverse frequency

# class_weights = torch.tensor(weights_np, dtype=torch.float)
# # after building torch tensor class_weights
# class_weights = class_weights / class_weights.mean()
# class_weights = torch.clamp(class_weights, 0.9, 1.2)  # softer spread

# class WeightedTrainer(Trainer):
#     def __init__(self, *args, class_weights=None, **kwargs):
#         super().__init__(*args, **kwargs)
#         # make sure we store a torch tensor
#         if isinstance(class_weights, np.ndarray):
#             class_weights = torch.tensor(class_weights, dtype=torch.float)
#         elif class_weights is not None and not isinstance(class_weights, torch.Tensor):
#             class_weights = torch.tensor(class_weights, dtype=torch.float)
#         self.class_weights = class_weights

#     # accept **kwargs for newer HF versions
#     def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
#         labels = inputs.pop("labels")
#         outputs = model(**inputs)
#         logits = outputs.logits

#         # ---- FIX: move weights to same device & dtype as logits ----
#         if self.class_weights is not None:
#             w = self.class_weights.to(device=logits.device, dtype=logits.dtype)
#         else:
#             w = None
#         # ------------------------------------------------------------

#         loss_fct = torch.nn.CrossEntropyLoss(weight=w, label_smoothing=0.00)
#         loss = loss_fct(logits, labels)

#         inputs["labels"] = labels  # optional: put back for HF internals
#         return (loss, outputs) if return_outputs else loss

# class SamplerWeightedTrainer(WeightedTrainer):
#     """Same as WeightedTrainer, but oversamples RIGHT in the train dataloader."""
#     def get_train_dataloader(self):
#         ds = self.train_dataset
#         # ds.y must be a torch.LongTensor of label ids (you have this in StanceDS)
#         y = ds.y
#         # STANCE2ID order: [neutral=0, left=1, right=2]
#         class_boost = torch.tensor([1.0, 1.0, 1.0], dtype=torch.float)  # tune 1.5–3.0
#         sample_weights = class_boost[y]
#         sampler = WeightedRandomSampler(sample_weights,
#                                         num_samples=len(sample_weights),
#                                         replacement=True)
#         return DataLoader(
#             ds,
#             batch_size=self.args.per_device_train_batch_size,
#             sampler=sampler,
#             collate_fn=self.data_collator,
#             pin_memory=True,
#             drop_last=self.args.dataloader_drop_last,
#         )

def compute_metrics(eval_pred):
    logits, y_true = eval_pred
    y_pred = logits.argmax(axis=-1)
    acc = accuracy_score(y_true, y_pred)
    p, r, f1, _ = precision_recall_fscore_support(
        y_true, y_pred, average="macro", zero_division=0
    )
    return {
        "accuracy": acc,
        "macro_precision": p,
        "macro_recall": r,
        "macro_f1": f1,
    }

# init trainining argument
args = TrainingArguments(
    "stance-clf",
    learning_rate=2e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=64,
    num_train_epochs=5,                 # allow up to 5 but early-stop
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="macro_f1",   # <-- change
    greater_is_better=True,             # <-- change
    warmup_ratio=0.01,
    weight_decay=0.005,
    max_grad_norm=1.0,
    report_to=[],
    logging_strategy="epoch",       # <-- add this
    logging_first_step=True
)

# Init trainer
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    tokenizer=tok,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)],  # optional
)
trainer.train()
trainer.save_model("stance-clf-best")
tok.save_pretrained("stance-clf-best")

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipython-input-1305527705.py:141: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy,Macro Precision,Macro Recall,Macro F1
1,0.962900,0.882614,0.543189,0.541578,0.576214,0.551556
2,0.837000,0.865170,0.579734,0.576892,0.613902,0.585152
3,0.726900,0.873546,0.573090,0.569504,0.603909,0.577025
4,0.628000,0.908838,0.573090,0.570500,0.606913,0.579595


('stance-clf-best/tokenizer_config.json',
 'stance-clf-best/special_tokens_map.json',
 'stance-clf-best/vocab.txt',
 'stance-clf-best/added_tokens.json',
 'stance-clf-best/tokenizer.json')

In [70]:
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np

out = trainer.predict(val_ds)
y_pred = np.argmax(out.predictions, axis=1)
y_true = out.label_ids

print(classification_report(
    y_true, y_pred,
    target_names=["neutral","left-leaning","right-leaning"],
    digits=2
))
print("Confusion matrix (rows=true, cols=pred):")
print(confusion_matrix(y_true, y_pred))

               precision    recall  f1-score   support

      neutral       0.52      0.78      0.62       116
 left-leaning       0.59      0.56      0.57       243
right-leaning       0.62      0.51      0.56       243

     accuracy                           0.58       602
    macro avg       0.58      0.61      0.59       602
 weighted avg       0.59      0.58      0.58       602

Confusion matrix (rows=true, cols=pred):
[[ 90  14  12]
 [ 44 136  63]
 [ 38  82 123]]
